In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import adam

import lightning as L
from torch.utils.data import TensorDataset, DataLoader

class LSTM(L.LightningModule):
    
    def __init__(self):
        # Create the weight and bias tensors
        super().__init__()
        
        # Use a normal distribution to randomly select initialisation values for our LSTM weights (torch.normal)
        mean = torch.tensor(0.0)
        std = torch.tensor(1.0)
        
        # requires_grad = True since we want to be able to optimise these weights
        
        # Weights for the Long Term Memory Remembering
        self.wlr1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.wlr2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.blr1 = nn.Parameter(torch.tensor(0.), requires_grad = True)
        
        # Weights for the potential new Long Term Memory to be remembered
        self.wpr1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.wpr2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.bpr1 = nn.Parameter(torch.tensor(0.), requires_grad = True)
        
        # Weights for the potential Long Term Memory output
        self.wp1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.wp2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.bp1 = nn.Parameter(torch.tensor(0.), requires_grad = True)
        
        # Weight for the output / short term memory
        self.wo1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.wo2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.bo1 = nn.Parameter(torch.tensor(0.), requires_grad = True)
        
    def unit(self, input_value, short_memory, long_memory):
        # Carry out the mathematical operations
        # Sigmoid functions utilised to determine % of values that will be remembered
        # Tanh functions utilised to determine the actual memory values
        
        long_remember_percent = torch.sigmoid((short_memory * self.wlr1) + (input_value * self.wlr2) + self.blr1)
        
        potential_remember_percent = torch.sigmoid((short_memory * self.wpr1) + (input_value * self.wpr2) + self.bpr1)
        potential_memory = torch.tanh((short_memory * self.wp1) + (input_value * self.wp2) + self.bp1)
        
        output_percent = torch.sigmoid((short_memory * self.wo1) + (input_value * self.wo2) + self.bo1)
        
        updated_long_memory = ((long_memory * long_remember_percent) + (potential_memory * potential_remember_percent))
        updated_short_memory = torch.tanh(updated_long_memory) * output_percent
        
        return ([updated_short_memory, updated_long_memory])
        
    def forward(self, input):
        # Carry out a forward pass through the unrolled LSTM unit
        
        long_memory = 0
        short_memory = 0
        
        day1 = input[0]
        day2 = input[1]
        day3 = input[2]
        day4 = input[3]
        
        # Cycle through the multiple days of data and unroll the LSTM units, while receiving the updated values for short and long term memory
        short_memory, long_memory = self.unit(day1, short_memory, long_memory)
        short_memory, long_memory = self.unit(day2, short_memory, long_memory)
        short_memory, long_memory = self.unit(day3, short_memory, long_memory)
        short_memory, long_memory = self.unit(day4, short_memory, long_memory)
        
        # Output value is the short memory
        return short_memory
         
    def configure_optimisers(self):
        # Configuring our optimiser - in this case ADAM
        return Adam(self.parameters())
        
    def training_step(self, batch, batch_idx):
        # Calculate the loss and the training progress
        input_i, label_i = batch
        output_i = self.forward(input_i[0])
        loss = (output_i - label_i) ** 2
        
        # Lightning Module that stores the training_loss values
        self.log("train_loss", loss)
        self.log(f"out_{label_i}", output_i)
        
        return loss
            
        

In [11]:
model = LSTM()
print("Untrained network - randomised weights predictions")
# We have to detach our predictions here si
print("Observed = 0, Predicted = ", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())

Untrained network - randomised weights predictions
Observed = 0, Predicted =  tensor(-0.2033)
